# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [3]:
Recommender = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
Recommender.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
Recommender = Recommender.groupby(['CustomerID','ProductName'])[['Quantity']].sum()
Recommender

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
customer_pivot = pd.pivot_table(Recommender, values='Quantity', index='ProductName',
                           columns='CustomerID',aggfunc=np.sum, fill_value=0)
customer_pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
similar_customers = pd.DataFrame(1/(1 + squareform(pdist(customer_pivot.T, 'euclidean'))),
                       index=customer_pivot.columns, columns=customer_pivot.columns)
similar_customers

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
similar_customers_2 = similar_customers[412].sort_values(ascending=False)[1:6]
similar_customers_2

CustomerID
3317    0.085638
3535    0.084959
1008    0.083333
1072    0.082709
639     0.082403
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
records_similar_cust = Recommender.loc[similar_customers_2.index]
records_similar_cust

Quantity
CustomerID ProductName                          
3317       Bay Leaf                            1
           Beef - Ground Medium                1
           Beef - Rib Eye Aaa                  1
           Butter - Unsalted                   1
           Cheese - Victor Et Berthold         1
...                                          ...
639        Veal - Inside, Choice               1
           Water, Tap                          1
           Wine - Magnotta - Belpaese          1
           Wine - Toasted Head                 2
           Wine - Two Oceans Cabernet          1

[272 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
purchase_records = records_similar_cust.groupby('ProductName').sum().sort_values(by='Quantity',ascending=False)
purchase_records

,Quantity
ProductName,
Olive - Spread Tapenade,3
Sprouts - Baby Pea Tendrils,3
Wine - Blue Nun Qualitatswein,3
"Veal - Inside, Choice",3
"Pepper - Black, Whole",3
...,...
Flour - Whole Wheat,1
Foam Cup 6 Oz,1
Garlic,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [14]:
top_products = customer_pivot.merge(purchase_records, left_on='ProductName', right_on='ProductName')
top_products = top_products.reset_index()
recommended_products = top_products.sort_values(by= 'Quantity', ascending = False)[['ProductName',200,'Quantity']].head(5)
recommended_products


,ProductName,200,Quantity
123,Olive - Spread Tapenade,1,3
175,Sprouts - Baby Pea Tendrils,0,3
201,Wine - Blue Nun Qualitatswein,0,3
193,"Veal - Inside, Choice",0,3
134,"Pepper - Black, Whole",1,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [31]:
def recommendations_for_users(customerID):

    recommended_list = similar_customers[customerID].sort_values(ascending=False)[1:6]
    
    recommended_similar_clients = records_similar_cust.loc[recommended_list.index]
    
    similar_purchases = recommended_similar_clients.groupby('ProductName').sum().sort_values(by= 'Quantity', ascending = False)
    
    r_f_u = customer_pivot.merge(similar_purchases, left_on='ProductName', right_on='ProductName')
    r_f_u =  r_f_u[ r_f_u[customerID]==0]
    r_f_u =  r_f_u.reset_index()
    possible_products = r_f_u.sort_values(by= 'Quantity', ascending = False)[['ProductName','Quantity']].head(5)
    return possible_products

In [32]:
customer_recomendations = {}
customers_ID = list(similar_customers.columns)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [33]:
for i in customers_ID:
    customer_recomendations[i] = recommendations_for_users(i)

In [37]:
customer_df = pd.DataFrame.from_dict(customer_recomendations, orient='index').reset_index()
customer_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\construction.py:309: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,index,0
0,33,ProductName Quantity ...
1,200,ProductName Quantity 81 ...
2,264,ProductName Quantity 50 ...
3,356,ProductName Quantity 74 ...
4,412,ProductName Quantity 1...
...,...,...
995,97928,"Empty DataFrame Columns: [ProductName, Quantit..."
996,98069,"Empty DataFrame Columns: [ProductName, Quantit..."
997,98159,"Empty DataFrame Columns: [ProductName, Quantit..."
998,98185,"Empty DataFrame Columns: [ProductName, Quantit..."


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [39]:
similar_customers = pd.DataFrame(1/(1 + squareform(pdist(customer_pivot.T, 'jaccard'))),
                       index=customer_pivot.columns, columns=customer_pivot.columns)
similar_customers

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.514644,0.529703,0.524038,0.513043,0.516129,0.510549,0.509174,0.511211,0.513889,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
200,0.514644,1.000000,0.518828,0.514523,0.503704,0.512000,0.520325,0.510204,0.518828,0.519313,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
264,0.529703,0.518828,1.000000,0.515695,0.527027,0.515419,0.510460,0.508772,0.515419,0.511013,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
356,0.524038,0.514523,0.515695,1.000000,0.517094,0.510917,0.508403,0.521531,0.510823,0.513393,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
412,0.513043,0.503704,0.527027,0.517094,1.000000,0.512605,0.516529,0.520362,0.510373,0.515284,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.518987,0.525581,0.515021,0.521951,0.520362,1.000000,0.524887,0.525346,0.511628,0.504000
98069,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.516129,0.512821,0.514644,0.511013,0.526549,0.524887,1.000000,0.527027,0.517510,0.523404
98159,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.512500,0.525822,0.520000,0.513889,0.515152,0.525346,0.527027,1.000000,0.524793,0.512605
